**Import Necessary Libraries**

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

**Define Functions for model and data preperation.**

In [3]:
def create_lstm_model(input_shape):
    model=Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='weighted_loss')
    return model

In [4]:
def weighted_loss(y_true, y_pred):
    weights = tf.range(1, tf.shape(y_true)[0]+1, dtype=tf.float32)
    #Exponential Weights
    weights = tf.math.exp(weights/ tf.reduce_max(weights))
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.reduce_sum(loss * weights) 

In [5]:
def prepare_data(df, sequence_length=60):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

    X, y = [],[]
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler